# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [150]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [151]:
cities_df = pd.read_csv('../cities.csv')
cities_df = cities_df.drop(['Unnamed: 0'], axis=1)
cities_df

,city,country,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
0,Port Alfred,ZA,-33.5906,26.8910,59.50,59.50,13.11,0,75
1,Chibombo,ZM,-14.6569,28.0706,60.66,60.66,9.37,0,66
2,Iqaluit,CA,63.7506,-68.5145,46.13,46.13,4.61,90,87
3,Rikitea,PF,-23.1203,-134.9692,74.70,74.70,12.64,0,74
4,Bam,IR,29.1060,58.3570,88.90,88.90,7.09,2,12
...,...,...,...,...,...,...,...,...,...
565,Tahoua,NE,14.8888,5.2692,84.18,84.18,4.61,20,65
566,Joshīmath,IN,30.5667,79.5667,66.34,66.34,1.81,31,82
567,Wamba,NG,8.9333,8.6000,72.10,72.10,2.46,98,95
568,Cienfuegos,CU,22.1461,-80.4356,89.74,94.01,2.30,40,66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [167]:
gmaps.configure(api_key = g_key)

locations = cities_df[["latitude", "longitude"]]
humidity = cities_df["humidity"]
humidity

100

In [168]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [154]:
hotel_df = cities_df.loc[(cities_df['max temperature']<80) & (cities_df['max temperature']>70) & (cities_df['wind speed']<10) & (cities_df['cloudiness']==0)]
hotel_df

,city,country,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
298,Marsá Maţrūḩ,EG,31.3525,27.2453,79.00,79.00,6.91,0,73
314,Jijila,RO,45.3000,28.1500,70.05,70.05,6.64,0,60
339,Chaman,PK,30.9236,66.4512,77.02,77.02,4.21,0,14
344,Goderich,CA,43.7501,-81.7165,70.45,75.20,9.22,0,60
359,Khorramabad,IR,33.4878,48.3558,78.48,78.48,0.63,0,16
361,Seoul,KR,37.5683,126.9778,72.46,76.44,5.75,0,73
382,Makat,KZ,47.6469,53.3463,76.89,76.89,4.90,0,18
385,Nhulunbuy,AU,-12.2333,136.7667,74.01,74.01,5.75,0,88
409,Barcelos,PT,41.5388,-8.6151,64.89,70.68,5.55,0,75
464,Oruzgan,AF,32.8333,66.0000,70.52,70.52,0.76,0,17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [155]:
hotel_df['Hotel Name'] = ""

<ipython-input-155-55c2a054e183>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


In [156]:
target_radius = 5000
target_search = "Hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": target_radius,
    "keyword": target_search,
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row['latitude']
    lon = row['longitude']

    params['location'] = f"{lat},{lon}"

    print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        print("------------")

        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("No hotel within 5000 meters")
        hotel_df.loc[index, 'Hotel Name'] = "No hotel within 5000 meters"
        print("------------")

Retrieving Results for Index 298.
Closest hotel is Arous El Bahr Hotel.
------------
Retrieving Results for Index 314.


C:\Users\Superpawn\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


No hotel within 5000 meters
------------
Retrieving Results for Index 339.
No hotel within 5000 meters
------------
Retrieving Results for Index 344.
Closest hotel is Samuels Hotel.
------------
Retrieving Results for Index 359.
Closest hotel is Gahar motel.
------------
Retrieving Results for Index 361.
Closest hotel is Novotel Ambassador Seoul Yongsan - Seoul Dragon City.
------------
Retrieving Results for Index 382.
Closest hotel is Gostinitsa Daulet.
------------
Retrieving Results for Index 385.
Closest hotel is Gove Peninsula Motel.
------------
Retrieving Results for Index 409.
Closest hotel is In Barcelos Hostel & Guest House.
------------
Retrieving Results for Index 464.
No hotel within 5000 meters
------------
Retrieving Results for Index 493.
Closest hotel is Sheraton Salta Hotel.
------------
Retrieving Results for Index 524.
Closest hotel is ASTON Kupang Hotel & Convention Center.
------------


In [157]:
hotel_df

,city,country,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity,Hotel Name
298,Marsá Maţrūḩ,EG,31.3525,27.2453,79.00,79.00,6.91,0,73,Arous El Bahr Hotel
314,Jijila,RO,45.3000,28.1500,70.05,70.05,6.64,0,60,No hotel within 5000 meters
339,Chaman,PK,30.9236,66.4512,77.02,77.02,4.21,0,14,No hotel within 5000 meters
344,Goderich,CA,43.7501,-81.7165,70.45,75.20,9.22,0,60,Samuels Hotel
359,Khorramabad,IR,33.4878,48.3558,78.48,78.48,0.63,0,16,Gahar motel
361,Seoul,KR,37.5683,126.9778,72.46,76.44,5.75,0,73,Novotel Ambassador Seoul Yongsan - Seoul Drago...
382,Makat,KZ,47.6469,53.3463,76.89,76.89,4.90,0,18,Gostinitsa Daulet
385,Nhulunbuy,AU,-12.2333,136.7667,74.01,74.01,5.75,0,88,Gove Peninsula Motel
409,Barcelos,PT,41.5388,-8.6151,64.89,70.68,5.55,0,75,In Barcelos Hostel & Guest House
464,Oruzgan,AF,32.8333,66.0000,70.52,70.52,0.76,0,17,No hotel within 5000 meters


In [163]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]
humidity = hotel_df["humidity"]

In [166]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=80,point_radius=3)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))